In [11]:
import pickle
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import warnings
import preprocessing_functions
import feature_engineering_functions
import pycaret
from pycaret.classification import *
from scipy.stats import pearsonr
from dataprep.eda import create_report
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler

Basic preprocessing actions (Fitbit)
1. Select experiment dates
2. Drop duplicates
3. Type conversion
4. One hot encoding

In [12]:
fitbit=pd.read_pickle('data/daily_fitbit_df_unprocessed.pkl')
fitbit=preprocessing_functions.fitbit_basic_preprocessing(fitbit)
fitbit=preprocessing_functions.fitbit_one_hot_encoding(fitbit)
fitbit

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,calories,filteredDemographicVO2Max,distance,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,scl_avg,resting_hr,sleep_duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,sleep_efficiency,sleep_deep_ratio,sleep_wake_ratio,sleep_light_ratio,sleep_rem_ratio,steps,minutes_in_default_zone_1,minutes_below_default_zone_1,minutes_in_default_zone_2,minutes_in_default_zone_3,age,gender,bmi,True,False,Aerobic Workout,Bike,Bootcamp,Circuit Training,Elliptical,Hike,Interval Workout,Martial Arts,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,0.866667,-1.788325,2351.59,62.79210,6517.5,71.701565,149.0,24.0,33.0,713.0,NaN,62.073070,31260000.0,0.0,445.0,76.0,0.0,93.0,1.243243,0.987013,0.921642,1.341772,8833.0,83.0,1349.0,0.0,0.0,0.0,0.0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,621e301e67b776a240608a72,2021-05-24,34.710232,NaN,NaN,NaN,NaN,74.0,0.700000,0.700,0.833333,-1.822489,3682.90,46.44026,8567.1,81.644017,227.0,8.0,3.0,729.0,NaN,61.390017,28380000.0,0.0,408.0,65.0,0.0,94.0,0.892157,0.928571,0.847826,0.922222,12441.0,164.0,997.0,49.0,0.0,0.0,1.0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e301367b776a24057738e,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2147.17,49.73933,5337.8,104.619583,61.0,3.0,43.0,1333.0,NaN,72.436947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7279.0,32.0,236.0,30.0,0.0,1.0,0.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,621e360b67b776a24039709f,2021-05-24,NaN,68.321,31.413,NaN,15.4,69.0,0.633333,0.650,0.800000,-2.055332,1723.61,45.23770,5028.5,77.061042,272.0,8.0,2.0,723.0,NaN,68.243822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7493.0,127.0,1306.0,7.0,0.0,1.0,1.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,621e2fce67b776a240279baa,2021-05-24,34.783747,83.170,15.941,96.6,15.2,70.0,0.733333,0.725,0.633333,-0.720949,2393.99,43.67853,2444.9,94.808800,143.0,0.0,0.0,796.0,NaN,85.806939,30060000.0,0.0,438.0,63.0,19.0,96.0,1.079545,1.145455,1.240741,0.663717,3317.0,701.0,735.0,0.0,0.0,0.0,0.0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959,621e345267b776a240691064,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1339.20,NaN,NaN,NaN,0.0,0.0,0.0,1440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4960,621e310d67b776a24003096d,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1252.80,NaN,NaN,NaN,0.0,0.0,0.0,1440.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4961,621e324e67b776a2400191cb,2022-01-17,33.104768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.469242,2150.69,44.99934,3079.0,NaN,176.0,4.0,5.0,579.0,NaN,63.885053,19320000.0,0.0,272.0,50.0,0.0,96.0,1.050000,0.961538,0.974093,0.381818,4278.0,NaN,NaN,NaN,NaN,1.0,0.0,2,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4962,621e30f467b776a240f22944,2022-01-17,34.352490,NaN,NaN,NaN,13.4,NaN,NaN,NaN,NaN,-2.346201,1864.23,60.17697,2233.9,63.148837,157.0,0.0,0.0,706.0,NaN,58.144251,29940000.0,0.0,421.0,78.0,0.0,94.0,0.701754,1.368421,1.103806,0.849315,3122.0,2.0,1348.0,0.0,0.0,0.0,0.0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Basic preprocessing actions (Sema)
1. Group positive and negative feelings

In [13]:
sema=pd.read_pickle('data/semas_read_from_the_base_experiment_dates.pkl')
sema = sema.sort_values(by='date', ascending=True)
sema=preprocessing_functions.sema_basic_preprocessing(sema)
sema

,id,date,negative_feelings,positive_feelings
0,621e2f3967b776a240c654db,2021-05-24,0,1
11,621e362467b776a2404ad513,2021-05-24,0,1
10,621e301e67b776a240608a72,2021-05-24,0,1
9,621e30e467b776a240e817c7,2021-05-24,1,0
8,621e36f967b776a240e5e7c9,2021-05-24,1,0
...,...,...,...,...
4932,621e30f467b776a240f22944,2022-01-16,0,1
4946,621e2f9167b776a240011ccb,2022-01-17,1,0
4944,621e346f67b776a24081744f,2022-01-17,1,0
4945,621e30f467b776a240f22944,2022-01-17,0,1


In [14]:
# merge fitbit and sema
data = pd.merge(fitbit, sema, on=['date','id'], how='right')
data

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,calories,filteredDemographicVO2Max,distance,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,scl_avg,resting_hr,sleep_duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,sleep_efficiency,sleep_deep_ratio,sleep_wake_ratio,sleep_light_ratio,sleep_rem_ratio,steps,minutes_in_default_zone_1,minutes_below_default_zone_1,minutes_in_default_zone_2,minutes_in_default_zone_3,age,gender,bmi,True,False,Aerobic Workout,Bike,Bootcamp,Circuit Training,Elliptical,Hike,Interval Workout,Martial Arts,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates,negative_feelings,positive_feelings
0,621e2f3967b776a240c654db,2021-05-24,NaN,69.820,28.594,NaN,16.4,76.0,0.800000,0.850,0.600000,-1.184004,1690.71,49.38003,958.5,72.790064,60.0,0.0,0.0,880.0,NaN,72.059965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1447.0,7.0,1162.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1
1,621e362467b776a2404ad513,2021-05-24,34.526732,67.118,37.722,95.3,10.6,0.0,0.000000,0.000,0.000000,-3.007565,2412.87,45.92161,5603.7,81.410428,182.0,1.0,7.0,784.0,NaN,74.830557,21480000.0,0.0,333.0,25.0,0.0,97.0,1.051724,0.531915,0.966507,0.673077,7818.0,138.0,1217.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1
2,621e301e67b776a240608a72,2021-05-24,34.710232,NaN,NaN,NaN,NaN,74.0,0.700000,0.700,0.833333,-1.822489,3682.90,46.44026,8567.1,81.644017,227.0,8.0,3.0,729.0,NaN,61.390017,28380000.0,0.0,408.0,65.0,0.0,94.0,0.892157,0.928571,0.847826,0.922222,12441.0,164.0,997.0,49.0,0.0,0.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
3,621e30e467b776a240e817c7,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3015.86,40.34114,4989.7,75.915034,89.0,15.0,32.0,1304.0,NaN,52.127587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6506.0,105.0,703.0,64.0,2.0,0.0,0.0,3.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4,621e36f967b776a240e5e7c9,2021-05-24,NaN,59.911,61.157,95.4,15.0,83.0,0.933333,0.775,0.800000,-2.530838,2508.62,37.35982,8152.2,77.542902,336.0,7.0,12.0,582.0,NaN,66.471270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11835.0,181.0,1259.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4943,621e30f467b776a240f22944,2022-01-16,33.503907,52.846,55.602,NaN,12.8,NaN,NaN,NaN,NaN,-2.560771,2912.83,60.14659,8663.9,78.629000,245.0,67.0,48.0,578.0,NaN,58.142721,30120000.0,0.0,439.0,63.0,0.0,98.0,0.216667,1.105263,1.430605,0.315789,13359.0,144.0,1295.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1
4944,621e2f9167b776a240011ccb,2022-01-17,34.716690,58.489,28.329,96.5,14.2,76.0,0.800000,0.600,0.933333,-2.096381,1913.30,42.74609,5484.0,75.568659,293.0,0.0,0.0,793.0,NaN,63.398194,21240000.0,0.0,307.0,47.0,0.0,96.0,0.666667,0.824561,0.804000,0.634615,7876.0,195.0,1104.0,4.0,0.0,1.0,1.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4945,621e346f67b776a24081744f,2022-01-17,32.903545,71.863,25.475,NaN,18.2,72.0,0.833333,0.625,0.733333,-3.234348,1549.08,NaN,1675.3,78.807015,113.0,0.0,0.0,761.0,NaN,74.639606,33960000.0,0.0,506.0,60.0,0.0,92.0,1.166667,0.845070,0.980769,1.016529,2528.0,56.0,1384.0,0.0,0.0,NaN,NaN,3.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4946,621e30f467b776a240f22944,2022-01-17,34.352490,NaN,NaN,NaN,13.4,NaN,NaN,NaN,NaN,-2.346201,

In [15]:
# merge fitbit and sema
data = pd.merge(fitbit, sema, on=['date','id'], how='inner')
data

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,calories,filteredDemographicVO2Max,distance,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,scl_avg,resting_hr,sleep_duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,sleep_efficiency,sleep_deep_ratio,sleep_wake_ratio,sleep_light_ratio,sleep_rem_ratio,steps,minutes_in_default_zone_1,minutes_below_default_zone_1,minutes_in_default_zone_2,minutes_in_default_zone_3,age,gender,bmi,True,False,Aerobic Workout,Bike,Bootcamp,Circuit Training,Elliptical,Hike,Interval Workout,Martial Arts,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates,negative_feelings,positive_feelings
0,621e301e67b776a240608a72,2021-05-24,34.710232,NaN,NaN,NaN,NaN,74.0,0.700000,0.700,0.833333,-1.822489,3682.90,46.44026,8567.1,81.644017,227.0,8.0,3.0,729.0,NaN,61.390017,28380000.0,0.0,408.0,65.0,0.0,94.0,0.892157,0.928571,0.847826,0.922222,12441.0,164.0,997.0,49.0,0.0,0.0,1.0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,621e362467b776a2404ad513,2021-05-24,34.526732,67.118,37.722,95.3,10.6,0.0,0.000000,0.000,0.000000,-3.007565,2412.87,45.92161,5603.7,81.410428,182.0,1.0,7.0,784.0,NaN,74.830557,21480000.0,0.0,333.0,25.0,0.0,97.0,1.051724,0.531915,0.966507,0.673077,7818.0,138.0,1217.0,0.0,0.0,1.0,0.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1
2,621e362467b776a2404ad513,2021-05-24,34.526732,67.118,37.722,95.3,10.6,0.0,0.000000,0.000,0.000000,-3.007565,2412.87,45.92161,5603.7,81.410428,182.0,1.0,7.0,784.0,NaN,74.830557,21480000.0,0.0,333.0,25.0,0.0,97.0,1.051724,0.531915,0.966507,0.673077,7818.0,138.0,1217.0,0.0,0.0,1.0,0.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1
3,621e362467b776a2404ad513,2021-05-24,34.526732,67.118,37.722,95.3,10.6,0.0,0.000000,0.000,0.000000,-3.007565,2412.87,45.92161,5603.7,81.410428,182.0,1.0,7.0,784.0,NaN,74.830557,21480000.0,0.0,333.0,25.0,0.0,97.0,1.051724,0.531915,0.966507,0.673077,7818.0,138.0,1217.0,0.0,0.0,1.0,0.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1
4,621e2f3967b776a240c654db,2021-05-24,NaN,69.820,28.594,NaN,16.4,76.0,0.800000,0.850,0.600000,-1.184004,1690.71,49.38003,958.5,72.790064,60.0,0.0,0.0,880.0,NaN,72.059965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1447.0,7.0,1162.0,0.0,0.0,1.0,0.0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4940,621e366567b776a24076a727,2022-01-16,33.277004,46.697,62.157,96.4,13.0,76.0,0.633333,0.825,0.800000,-2.266232,3800.45,NaN,14638.6,65.622004,363.0,65.0,46.0,517.0,NaN,49.959379,26940000.0,0.0,372.0,77.0,0.0,94.0,0.828947,1.833333,1.403315,0.723684,21413.0,48.0,1285.0,6.0,0.0,1.0,0.0,2,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4941,621e346f67b776a24081744f,2022-01-17,32.903545,71.863,25.475,NaN,18.2,72.0,0.833333,0.625,0.733333,-3.234348,1549.08,NaN,1675.3,78.807015,113.0,0.0,0.0,761.0,NaN,74.639606,33960000.0,0.0,506.0,60.0,0.0,92.0,1.166667,0.845070,0.980769,1.016529,2528.0,56.0,1384.0,0.0,0.0,NaN,NaN,3,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4942,621e331067b776a24085dd3f,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1120.51,NaN,NaN,NaN,0.0,0.0,0.0,1259.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
4943,621e2f9167b776a240011ccb,2022-01-17,34.716690,58.489,28.329,96.5,14.2,76.0,0.800000,0.600,0.933333,-2.096381,1913.30,42.74609,

In [ ]:
data = data.drop(columns=['positive_feelings'])
data

Preprocessing steps added:
1. drop 'spo2', 'scl_avg'
2. replace outliers and NaNs with columns median
3. date engineering

In [ ]:
data = preprocessing_functions.post_preprocessing(data, isSema=True)
data

# Adds wear_day

In [ ]:
data['wear_day'] = data.apply(feature_engineering_functions.f, axis=1)
data

In [ ]:
data.columns

# Adds stress_quantile

In [ ]:
data["stress_quantile"] = pd.qcut(data["stress_score"].rank(method='first'), [0, .25, .75, 1], labels=["low", "medium", "high"])
data

In [ ]:
sns.countplot(y="stress_quantile", data=data)

In [ ]:
data['stress_quantile'].replace(to_replace=['low', 'medium', 'high'], value=[0, 1, 2], inplace=True)
data

In [ ]:
sns.countplot(y="stress_quantile", data=data)

# Adds sleep duration average values for each user

In [ ]:
data

In [ ]:
data['sleep_repetition'] = data['id'].map(data.groupby('id')['sleep_duration'].unique())
data['average_sleep_duration']=pd.DataFrame(data['sleep_repetition'].values.tolist()).mean(1)
data = data.drop(columns=['sleep_repetition'])

In [ ]:
train_data, test_data = preprocessing_functions.train_test_split_per_user(data)

fold_groups = train_data.id
train_data = train_data.drop(columns=['id'])
test_data = test_data.drop(columns=['id'])

In [ ]:
grid = setup(data=train_data, target='negative_feelings', silent=True, session_id=123, fold_strategy='groupkfold',
              fold=3,  fold_groups=fold_groups, test_data=test_data, normalize=True, normalize_method='minmax', transformation=True, feature_interaction = True, feature_ratio = True, fix_imbalance=True)

In [ ]:
best=compare_models()

In [ ]:
best

In [ ]:
plot_model(best, plot='auc')

In [ ]:
plot_model(best, plot='feature')